In [1]:
%matplotlib ipympl
import torch.nn as nn
import matplotlib
matplotlib.rcParams['animation.embed_limit'] = 1000
from tqdm.auto import tqdm
from IPython.display import HTML
import torch
import numpy as np
import torch.nn.functional as F
import matplotlib.pyplot as plt
plt.rcParams['animation.html'] = 'jshtml'
from matplotlib.animation import FuncAnimation

class SymmetricEigenvalueNet(nn.Module):
    def __init__(self, matrix_size, hidden_layers=[128, 256, 128]):
        super(SymmetricEigenvalueNet, self).__init__()
        input_size = (matrix_size * (matrix_size + 1)) // 2
        self.input_layer = nn.Linear(input_size, hidden_layers[0])
        self.input_norm = nn.LayerNorm(hidden_layers[0])
        self.hidden_layers = nn.ModuleList()

        for i in range(len(hidden_layers) - 1):
            self.hidden_layers.append(nn.Linear(hidden_layers[i], hidden_layers[i + 1]))
            self.hidden_layers.append(nn.LayerNorm(hidden_layers[i + 1]))
            self.hidden_layers.append(nn.ReLU())

        self.output_layer = nn.Linear(hidden_layers[-1], matrix_size)
        self.apply(self._init_weights)

    @staticmethod
    def _init_weights(module):
        if isinstance(module, nn.Linear):
            nn.init.xavier_normal_(module.weight)
            if module.bias is not None:
                nn.init.zeros_(module.bias)

    def forward(self, x):
        x_flat = self.extract_lower_triangular(x)
        x = self.input_layer(x_flat)
        x = self.input_norm(x)
        x = F.relu(x)

        for i in range(0, len(self.hidden_layers), 3):
            x = self.hidden_layers[i](x)
            x = self.hidden_layers[i + 1](x)
            x = self.hidden_layers[i + 2](x)

        return self.output_layer(x)

    @staticmethod
    def extract_lower_triangular(matrix):
        if matrix.dim() == 2:
            matrix = matrix.unsqueeze(0)
        batch_size, n, _ = matrix.shape
        mask = torch.tril(torch.ones(n, n, device=matrix.device, dtype=torch.bool))
        lower_tri = matrix[mask.repeat(batch_size, 1, 1)]
        return lower_tri.view(batch_size, -1)

def generate_sparse_symmetric_matrix(size=5, k=None, seed=42, sparse_format=False):
    """
    Generate a sparse symmetric matrix with k non-zero elements and no zero eigenvalues
    """
    np.random.seed(seed)
    torch.manual_seed(seed)

    if k is None:
        k = size  # Default number of non-zero elements

    # Ensure k is not larger than the number of elements in the matrix
    max_elements = (size * (size + 1)) // 2  # Number of elements in upper triangular part
    k = min(k, max_elements)

    # Initialize zero matrix
    matrix = np.zeros((size, size))

    # Generate random positions for non-zero elements in upper triangular part
    positions = np.random.choice(max_elements, k, replace=False)

    # Convert linear indices to matrix indices
    for pos in positions:
        i, j = np.triu_indices(size, 0)[0][pos], np.triu_indices(size, 0)[1][pos]
        # Generate random value ensuring diagonal elements are positive
        if i == j:
            val = np.random.uniform(1, 2)  # Ensure positive diagonal
        else:
            val = np.random.uniform(-1, 1)
        matrix[i, j] = val
        matrix[j, i] = val  # Ensure symmetry

    # Add a positive constant to diagonal to ensure no zero eigenvalues
    min_eigenval = np.min(np.linalg.eigvals(matrix))
    if min_eigenval <= 0:
        shift = -min_eigenval + 0.1
        np.fill_diagonal(matrix, matrix.diagonal() + shift)

    if not sparse_format:
        return torch.tensor(matrix, dtype=torch.float32)

    # Convert to sparse format
    sparse_list = []
    for i in range(size):
        for j in range(size):
            if matrix[i, j] != 0:
                sparse_list.append([i + 1, j + 1, matrix[i, j]])
    return sparse_list


def print_matrix_representations(matrix_size=2, k=None, seed=42):
    # Generate both representations
    full_matrix = generate_sparse_symmetric_matrix(size=matrix_size, k=k, seed=seed, sparse_format=False)
    sparse_matrix = generate_sparse_symmetric_matrix(size=matrix_size, k=k, seed=seed, sparse_format=True)

    print(f"\nMatrix Size: {matrix_size}x{matrix_size}")
    print(f"Number of non-zero elements: {len(sparse_matrix)//2 + sum(1 for x in sparse_matrix if x[0] == x[1])}")

    print("\nFull Matrix Representation:")
    print(full_matrix)

    print("\nSparse Matrix Representation (row, column, value):")
    for entry in sparse_matrix:
        print(f"({int(entry[0])}, {int(entry[1])}) -> {entry[2]:.4f}")

    eigenvals = np.linalg.eigvals(full_matrix.numpy())
    print("\nEigenvalues:")
    print(eigenvals)


def generate_true_eigenvalues(mat):
    return torch.tensor(np.linalg.eigvals(mat.numpy()), dtype=torch.float32)

def custom_eigenvalue_loss(predicted_eigenvalues, true_batch_eigenvalues):
    return F.mse_loss(predicted_eigenvalues, true_batch_eigenvalues)

def generate_symmetric_matrix(size=5,
                              seed=42,
                              sparse_format=False):
    np.random.seed(seed)
    torch.manual_seed(seed)
    matrix = np.random.rand(size, size)
    symmetric_matrix = (matrix + matrix.T) / 2

    if not sparse_format:
        return torch.tensor(symmetric_matrix, dtype=torch.float32)

    # Convert to sparse format
    sparse_list = []
    for i in range(size):
        for j in range(size):
            sparse_list.append([i + 1, j + 1, symmetric_matrix[i, j]])
    return sparse_list

def create_animation(train_matrices, true_eigenvalues, test_matrix, epochs=300, batch_size=32, filename='eigenvalues_animation.mp4'):
    model = SymmetricEigenvalueNet(matrix_size=train_matrices.shape[1])
    optimizer = torch.optim.AdamW(model.parameters(), lr=0.001, weight_decay=1e-4)
    scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='min', factor=0.5, patience=50)

    test_true_eigenvalues = generate_true_eigenvalues(test_matrix)

    fig = plt.figure(figsize=(12, 6))
    plt.subplots_adjust(right=0.85)
    ax = fig.add_subplot(111)

    colors = plt.cm.rainbow(np.linspace(0, 1, train_matrices.shape[1]))
    lines = []
    true_lines = []

    for i, color in enumerate(colors):
        line, = ax.plot([], [], label=f'Predicted λ{i + 1}', color=color)
        lines.append(line)
        true_line, = ax.plot([], [], '--', label=f'True λ{i + 1}', color=color, alpha=0.5)
        true_lines.append(true_line)

    ax.set_xlabel('Epoch')
    ax.set_ylabel('Eigenvalues')
    ax.set_title('Eigenvalue Prediction Evolution During Training')
    ax.grid(True)
    ax.legend(bbox_to_anchor=(1, 1), loc='upper left')

    epochs_list = []
    eigenvalues_history = []

    def init():
        for line in lines + true_lines:
            line.set_data([], [])
        return lines + true_lines

    progress_bar = tqdm(total=epochs, desc="Creating animation")

    def animate(frame):
        epoch = frame
        epoch_loss = 0.0
        model.train()

        train_dataset = torch.utils.data.TensorDataset(train_matrices, true_eigenvalues)
        train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=batch_size, shuffle=True)

        for batch_matrices, batch_eigenvalues in train_loader:
            optimizer.zero_grad()
            predicted_eigenvalues = model(batch_matrices)
            loss = custom_eigenvalue_loss(predicted_eigenvalues, batch_eigenvalues)
            loss.backward()
            optimizer.step()
            epoch_loss += loss.item()

        avg_epoch_loss = epoch_loss / len(train_loader)
        scheduler.step(avg_epoch_loss)

        model.eval()
        with torch.no_grad():
            test_pred = model(test_matrix.unsqueeze(0))[0]

        epochs_list.append(epoch)
        eigenvalues_history.append(test_pred.numpy())

        for i, line in enumerate(lines):
            line.set_data(epochs_list, [ev[i] for ev in eigenvalues_history])
            true_lines[i].set_data([0, epoch], [test_true_eigenvalues[i], test_true_eigenvalues[i]])

        progress_bar.update(1)
        ax.relim()
        ax.autoscale_view()
        return lines + true_lines

    anim = FuncAnimation(fig, animate, init_func=init, frames=epochs, interval=100, blit=True)

    # Save animation to MP4 file
    anim.save(filename, writer='ffmpeg', fps=30)
    plt.close()

    # Clear memory
    del train_matrices, true_eigenvalues, model, optimizer, scheduler
    torch.cuda.empty_cache() if torch.cuda.is_available() else None

    # Display video with autoplay and loop
    return HTML(f"""
    <video width="800" height="400" controls autoplay loop>
        <source src="{filename}" type="video/mp4">
    </video>
    """)

# Usage example:
matrix_size = 5
num_nonzero = 7
num_train_matrices = matrix_size * 1000

# Print both matrix representations
print_matrix_representations(matrix_size=matrix_size, k=num_nonzero, seed=42)

# Generate training data
train_matrices = torch.stack([generate_sparse_symmetric_matrix(size=matrix_size, k=num_nonzero, seed=i)
                            for i in range(num_train_matrices)])
test_matrix = generate_sparse_symmetric_matrix(size=matrix_size, k=num_nonzero, seed=42)
true_eigenvalues = torch.stack([generate_true_eigenvalues(matrix) for matrix in train_matrices])

# Create and display animation
animation_display = create_animation(train_matrices, true_eigenvalues, test_matrix, epochs=1000)
display(animation_display)



Matrix Size: 5x5
Number of non-zero elements: 11

Full Matrix Representation:
tensor([[ 1.8751,  0.0000,  0.9844,  0.0000,  0.0000],
        [ 0.0000,  2.7572,  0.0000,  0.0000, -0.9984],
        [ 0.9844,  0.0000,  1.9615,  0.0000,  0.3018],
        [ 0.0000,  0.0000,  0.0000,  0.8187,  0.4440],
        [ 0.0000, -0.9984,  0.3018,  0.4440,  0.8187]])

Sparse Matrix Representation (row, column, value):
(1, 1) -> 1.8751
(1, 3) -> 0.9844
(2, 2) -> 2.7572
(2, 5) -> -0.9984
(3, 1) -> 0.9844
(3, 3) -> 1.9615
(3, 5) -> 0.3018
(4, 4) -> 0.8187
(4, 5) -> 0.4440
(5, 2) -> -0.9984
(5, 3) -> 0.3018
(5, 4) -> 0.4440
(5, 5) -> 0.8187

Eigenvalues:
[3.2206936  2.894711   0.09999999 1.1099192  0.9059134 ]


Creating animation:   0%|          | 0/1000 [00:00<?, ?it/s]